<img src="https://s3.cloud.cmctelecom.vn/tinhte1/2018/03/4267082_CV.jpg" width=browser_width >

# **Tiki Web Scraping with Selenium**


**Overview**: Build a web-crawler that take in a Tiki URL and return a dataframe 

**Due Date**: Before Monday next week.

**Requirements** 
1. Your function should be able to take in an URL and return a pandas dataframe
2. The final dataframe should at least contain the following informations: 
    * Product Name
    * Price
    * URL of the product image
    * URL of that product page

Try to follow the guideline below

#Install resources

In [ ]:
# install selenium and other resources for crawling data
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic

# Import necessary libraries

In [ ]:
import re
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

#Configuration for Driver and links

In [ ]:
# Urls
TIKI = 'https://tiki.vn'
MAIN_CATEGORIES = [
    {'Name': 'Điện Thoại - Máy Tính Bảng',
     'URL': 'https://tiki.vn/dien-thoai-may-tinh-bang/c1789?src=c.1789.hamburger_menu_fly_out_banner'},

    {'Name': 'Điện Tử - Điện Lạnh',
     'URL': 'https://tiki.vn/tivi-thiet-bi-nghe-nhin/c4221?src=c.4221.hamburger_menu_fly_out_banner'},

    {'Name': 'Phụ Kiện - Thiết Bị Số', 
     'URL': 'https://tiki.vn/thiet-bi-kts-phu-kien-so/c1815?src=c.1815.hamburger_menu_fly_out_banner'},

    {'Name': 'Laptop - Thiết bị IT', 
     'URL': 'https://tiki.vn/laptop-may-vi-tinh/c1846?src=c.1846.hamburger_menu_fly_out_banner'},

    {'Name': 'Máy Ảnh - Quay Phim', 
     'URL': 'https://tiki.vn/may-anh/c1801?src=c.1801.hamburger_menu_fly_out_banner'},

    {'Name': 'Điện Gia Dụng', 
     'URL': 'https://tiki.vn/dien-gia-dung/c1882?src=c.1882.hamburger_menu_fly_out_banner'},

    {'Name': 'Nhà Cửa Đời Sống', 
     'URL': 'https://tiki.vn/nha-cua-doi-song/c1883?src=c.1883.hamburger_menu_fly_out_banner'},

    {'Name': 'Hàng Tiêu Dùng - Thực Phẩm', 
     'URL': 'https://tiki.vn/bach-hoa-online/c4384?src=c.4384.hamburger_menu_fly_out_banner'},

    {'Name': 'Đồ chơi, Mẹ & Bé', 
     'URL': 'https://tiki.vn/me-va-be/c2549?src=c.2549.hamburger_menu_fly_out_banner'},

    {'Name': 'Làm Đẹp - Sức Khỏe', 
     'URL': 'https://tiki.vn/lam-dep-suc-khoe/c1520?src=c.1520.hamburger_menu_fly_out_banner'},

    {'Name': 'Thể Thao - Dã Ngoại', 
     'URL': 'https://tiki.vn/the-thao/c1975?src=c.1975.hamburger_menu_fly_out_banner'},

    {'Name': 'Xe Máy, Ô tô, Xe Đạp', 
     'URL': 'https://tiki.vn/o-to-xe-may-xe-dap/c8594?src=c.8594.hamburger_menu_fly_out_banner'},

    {'Name': 'Hàng quốc tế', 
     'URL': 'https://tiki.vn/hang-quoc-te/c17166?src=c.17166.hamburger_menu_fly_out_banner'},

    {'Name': 'Sách, VPP & Quà Tặng', 
     'URL': 'https://tiki.vn/nha-sach-tiki/c8322?src=c.8322.hamburger_menu_fly_out_banner'},

    {'Name': 'Voucher - Dịch Vụ - Thẻ Cào', 
     'URL': 'https://tiki.vn/voucher-dich-vu/c11312?src=c.11312.hamburger_menu_fly_out_banner'}
]

# Global driver to use throughout the script
DRIVER = None

# 0. Function to Start and Close Driver

In [ ]:
# Wrapper to close driver if its created
def close_driver():
    global DRIVER
    if DRIVER is not None:
        DRIVER.close()
    DRIVER = None

# Function to (re)start driver
def start_driver(force_restart=False):
    global DRIVER
    
    if force_restart:
        close_driver()
    
    # Setting up the driver
    options = webdriver.ChromeOptions()
    options.add_argument('-headless') # we don't want a chrome browser opens, so it will run in the background
    options.add_argument('-no-sandbox')
    options.add_argument('-disable-dev-shm-usage')

    DRIVER = webdriver.Chrome('chromedriver',options=options)

In [ ]:
close_driver()

In [ ]:
start_driver(force_restart=True)

In [ ]:
DRIVER.get('https://tiki.vn/thiet-bi-kts-phu-kien-so/c1815?src=c.1815.hamburger_menu_fly_out_banner')

In [ ]:
DRIVER.current_url

'https://tiki.vn/thiet-bi-kts-phu-kien-so/c1815?src=c.1815.hamburger_menu_fly_out_banner'

In [ ]:
all_news_elements = DRIVER.find_elements(By.CLASS_NAME, 'product-item')

In [ ]:
len(all_news_elements)

48

# 1. Function to get info from one product

>**NOTE:** Sometimes, the web element returned by the driver can be faulty due to the way the website is set up. This can lead to a situation where calling `.text` from that web element returns an empty string, even though there are visible texts inside the element when checked manually using Inspect. When this happens, you can use `.get_attribute('innerHTML')` instead of `.text`.

In [ ]:
all_news_elements = DRIVER.find_elements(By.CLASS_NAME, 'product-item')
print(all_news_elements)

[<selenium.webdriver.remote.webelement.WebElement (session="18605115fc4fc1613bee232252992dba", element="a6f95667-ea47-4297-89b4-0857fbdcf877")>, <selenium.webdriver.remote.webelement.WebElement (session="18605115fc4fc1613bee232252992dba", element="894ce046-1763-4ad7-b9df-06ace893ba29")>, <selenium.webdriver.remote.webelement.WebElement (session="18605115fc4fc1613bee232252992dba", element="6decd252-22a3-478b-a377-f40a5c3da874")>, <selenium.webdriver.remote.webelement.WebElement (session="18605115fc4fc1613bee232252992dba", element="242a3f97-a168-4f84-b113-c90b7287ed61")>, <selenium.webdriver.remote.webelement.WebElement (session="18605115fc4fc1613bee232252992dba", element="55ce077a-328c-4224-b3ad-f3c48abd5ba0")>, <selenium.webdriver.remote.webelement.WebElement (session="18605115fc4fc1613bee232252992dba", element="ffd2b155-f620-4b7e-9f87-0c827c3f31f6")>, <selenium.webdriver.remote.webelement.WebElement (session="18605115fc4fc1613bee232252992dba", element="43ed680a-8a2c-4463-b111-6b8879b1

In [ ]:
len(all_news_elements)

48

The first page of Tiki has 48 all news element and needed to scrape information from 48 elements as known as product items

In [ ]:
# Function to extract product info from the product
def get_product_info_single(news_element):
    info = {'name':'',
            'price':'',
            'discount':'',
            'product_url':'',
            'image':'',
            'freeship':'',
            'rating':'',
            'badge_rhht':'',
            'freegift':'',
            'installment':''}

    # name get name through find_element_by_class_name
    try:
        info['name'] = news_element.find_element(By.CLASS_NAME, 'name').find_element(By.TAG_NAME,'span').get_attribute('innerHTML')
    except NoSuchElementException:
        pass

    # get price find_element_by_class_name
    try:
        info['price'] = news_element.find_element(By.CLASS_NAME, 'price-discount__price').get_attribute('innerHTML')
    except NoSuchElementException:
        info['price'] = -1

    # get discount find_element_by_class_name
    try:
        info['discount'] = news_element.find_element(By.CLASS_NAME, 'price-discount__discount').get_attribute('innerHTML')
    except NoSuchElementException:
        info['discount'] = False
    
    # get link from .get_attribute()
    try:
        product_link     = news_element.get_attribute('href')
        info['product_url'] = news_element.get_attribute('href')
    except NoSuchElementException:
        pass

    # get thumbnail by class_name and Tag name and get_attribute()
    try:
        thumbnail = news_element.find_element(By.CLASS_NAME, 'webpimg-container')
        info['image'] = thumbnail.find_element_by_tag_name('img').get_attribute('src')
    except NoSuchElementException:
        pass

    # get freeship
    try:
        thumbnail = news_element.find_element(By.CLASS_NAME, 'thumbnail')
        info['freeship'] = thumbnail.find_element_by_tag_name('img').get_attribute('src')
    except NoSuchElementException:
        pass

      # get number of ratings  
    try:
        elem_review = news_element.find_element(By.CLASS_NAME, 'average')
        info['rating'] = float(re.sub(r'\D','',elem_review.get_attribute('style')))/100*5
    except NoSuchElementException:
        info['rating']=0
        pass

          # get the badgge of re hon hoan tien  
    try:
        thumbnail = news_element.find_element(By.CLASS_NAME, 'item')
        info['badge_rhht'] = thumbnail.find_element_by_tag_name('img').get_attribute('src')
    except NoSuchElementException:
        info['badge_rhht']= False
        pass

          # get the badge of free gift
    try:
        fg = news_element.find_element(By.CLASS_NAME, 'freegift_list')
        info['freegift'] = fg.find_element_by_tag_name('img').get_attribute('src')
    except NoSuchElementException:
        info['freegift']= False
        pass

          # get the badge of installment
    try:
        fg = news_element.find_element(By.CLASS_NAME, 'badge_benefits')
        info['freegift'] = fg.find_element_by_tag_name('img').get_attribute('src')
    except NoSuchElementException:
        info['freegift']= False
        pass

    
    return info


In [ ]:
get_product_info_single(news_element)

/usr/local/lib/python3.7/dist-packages/selenium/webdriver/remote/webelement.py:341: UserWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  warnings.warn("find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead")


{'badge_rhht': 'https://salt.tikicdn.com/ts/upload/51/ac/cc/528e80fe3f464f910174e2fdf8887b6f.png',
 'discount': '-7%',
 'freegift': False,
 'freeship': 'https://salt.tikicdn.com/ts/upload/dc/0d/49/3251737db2de83b74eba8a9ad6d03338.png',
 'image': 'https://salt.tikicdn.com/cache/200x200/ts/product/8b/7b/4d/b552547d3f21139e0bf413c60d7d7105.jpg',
 'installment': '',
 'name': 'Tai nghe chụp tai không dây Remax RB-660HB - Hàng chính hãng',
 'price': '649.000 ₫',
 'product_url': 'https://tka.tiki.vn/pixel/pixel?data=djAwMVbUpo68pL4ucy-mvXwOjzxgXqFqhO_amKZ856A7uNzDbs26_awrhjP96tXjdOC3UWFSUV5NFHWPdEX4MR2eqv0x_6Au96gv6FKCm5tg438nQsMhrzN86ySAsPg2uKpWJVs25zcLaPUlvOrrdpFRsPHYjlYDqzSQfLCianNco_IkA8cTZkFMJF6ogLc2_1h9YGLETd6OjfncG_xVsZyfiKDolyBzjyWduQNTEPZVqTfPBRpJAfU6HdTgYk-lw961sT7ie52syFYbBDVzRKlxGhA3svJC-7Mpsh7Nkisv2WPTdqmSk5Y8nCLgoHkJh56h1zo8LPpMoIyhgB5H1liX9CLBtSefslYPKwaSRBEOZbgtuM7wRXgI-d65mAfRKpG9ZHXvu7KJYcX13U-nPJSJjjsKHoTJuGIWCtDl-oREHisTFpxBuW3YsN-RLZDXZJLDBghNIcIObizvgZ4J-xBcb1wDkG0nUtvWt

In [ ]:
# product_element = news_element.find_element(By.CLASS_NAME, 'name').find_element(By.TAG_NAME,'span').get_attribute('innerHTML')
# print(product_element)

# price_number = news_element.find_element(By.CLASS_NAME, 'price-discount__price').get_attribute('innerHTML')
# print(price_number)

# price=b_element.get_attribute('innerHTML')
# print(priced)
# a_element = description_element.find_element_by_tag_name('span')
# print(product_name)


1.290.000 ₫


In [ ]:
news_element.find_element(By.CLASS_NAME, 'price-discount__price').get_attribute('innerHTML')

'74.000 ₫'

In [ ]:
news_element.find_element(By.CLASS_NAME, 'price-discount__discount').get_attribute('innerHTML')

'-20%'

In [ ]:
product_page= news_element.get_attribute('href')
print(product_page)


https://tka.tiki.vn/pixel/pixel?data=djAwMXn-sbjBTSe8D5tZnTx1DRNN7JC6-OW3V6YKv4F1VZkGNQU3b4lPNtqYJqn1eAsaJCF8vFyk6YuR5adeliwJhBkNwjG-O48nZyQ9Th8EGmp2JAz2l8HTk-RsXccK6SbEk3PDgSkoLJfX_NNYrYvUQG6wYbs_YluNrEzECcfOpazfx0XqZRS1YNOAPvgqHYbeduHPXqnnfJrM1HHuUMNvRsm1r1egHsrvMqbjXYQ5DKg2RxNqt8FC9B-X0OhMBH9Wa2wRc-IpeyxnTGhSZ1t1s_WJtWzpk38qbEoDDpRZOc4Gi0s7PsUrVwzcwrvVPyasH6X7qcESGmsA2xN2JrGbHNhdxbBVxML1rPjQIz6ze4xSRH1BczG_473XjJCZoIv-U0Fg_dT_fMTci-W3d4lSdVwI8wwA2oL_qvoAaAtBd8cc-u9z6gTQwDr_tC4kO8828tyrwPgLUv__Fzi4RrOMc5Gbb0szxWcbLVXpStN_5o_zDUvYFG75O0-HelldqYolY4VxMkQSmPX-leGesChI-9PEeFHe-_Wr7p20UQTKKZntHE1UyAzBIqs5Dq_s1e1gzLJvE-WBtwU_cqhM0M-fUEtpt_rwcMwobH28zbI2mjzN_ID0y-Quq3wDSRmzTZfVq1ARFj2DRjC3AfMz2saxc0AM8NaaS3vpzBidvYdYZq6GgdkmrDN9T6U3dXoxzJCObXaKHrlE8ESkZReYB1jv-AwA-KEO4wTumEkJQw7gJGFjTn9qFTJanwTw7O7piQk-DN9U0Zp_0h-CkPTPg3YguF7-2afkEsojCjNbK2jo0294XnrY7uRp8sNDdSD5KZ0kiBSjGaw81At9J2CTWhRRyWONNlW-tsPZzS9yqIpRYTaA3ebEq1YXLk10HdRkPtZThFaJpemRPBj18rn0dsDbePhBPhC9aYEFa73PbWyl6QxtyOqGsrPIi2wkKFcd7FC

In [ ]:

thumbnail_link = news_element.find_element(By.CLASS_NAME, 'webpimg-container').find_element_by_tag_name('img').get_attribute('src')
print(thumbnail_link)


https://salt.tikicdn.com/cache/200x200/ts/product/9d/71/3a/362d7f1daaf81f32919f7a4d28c72d40.jpg


/usr/local/lib/python3.7/dist-packages/selenium/webdriver/remote/webelement.py:341: UserWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  warnings.warn("find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead")


In [ ]:
thumbnail = news_element.find_element(By.CLASS_NAME, 'thumbnail').find_element_by_tag_name('img').get_attribute('src')
print(thumbnail)

https://salt.tikicdn.com/ts/upload/dc/0d/49/ef9dc5d8164bd62b011e54276502b342.png


/usr/local/lib/python3.7/dist-packages/selenium/webdriver/remote/webelement.py:341: UserWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  warnings.warn("find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead")


In [ ]:
t= news_element.find_element(By.CLASS_NAME, 'average').get_attribute('innerHTML')
print(t)

<svg stroke="currentColor" fill="currentColor" stroke-width="0" viewBox="0 0 24 24" size="14" color="#fdd836" height="14" width="14" xmlns="http://www.w3.org/2000/svg" style="color: rgb(253, 216, 54);"><path d="M12 17.27L18.18 21l-1.64-7.03L22 9.24l-7.19-.61L12 2 9.19 8.63 2 9.24l5.46 4.73L5.82 21z"></path></svg><svg stroke="currentColor" fill="currentColor" stroke-width="0" viewBox="0 0 24 24" size="14" color="#fdd836" height="14" width="14" xmlns="http://www.w3.org/2000/svg" style="color: rgb(253, 216, 54);"><path d="M12 17.27L18.18 21l-1.64-7.03L22 9.24l-7.19-.61L12 2 9.19 8.63 2 9.24l5.46 4.73L5.82 21z"></path></svg><svg stroke="currentColor" fill="currentColor" stroke-width="0" viewBox="0 0 24 24" size="14" color="#fdd836" height="14" width="14" xmlns="http://www.w3.org/2000/svg" style="color: rgb(253, 216, 54);"><path d="M12 17.27L18.18 21l-1.64-7.03L22 9.24l-7.19-.61L12 2 9.19 8.63 2 9.24l5.46 4.73L5.82 21z"></path></svg><svg stroke="currentColor" fill="currentColor" stroke-widt

# 2. Function to scrape info of all products from a Page URL

To make your own life easier, you should use the function for a single product inside this one.

In [ ]:
# Function to scrape all products from a page
def get_product_info_from_page(page_url):
   
    global DRIVER

    data = []            # Store the info dictionary of each product in this list
    DRIVER.get(page_url) # Use the driver to get info from the product page
    time.sleep(2)        # MUST have the sleep function

    products_all = DRIVER.find_elements(By.CLASS_NAME, 'product-item')
    print(type(len(products_all)))
    print(f'Found {len(products_all)} products')

    for product in products_all:
      result = get_product_info_single(product) 
      data.append(result)
    return data

In [ ]:
get_product_info_from_page('https://tiki.vn/thiet-bi-kts-phu-kien-so/c1815?src=c.1815.hamburger_menu_fly_out_banner')

<class 'int'>
Found 48 products


/usr/local/lib/python3.7/dist-packages/selenium/webdriver/remote/webelement.py:341: UserWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  warnings.warn("find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead")


[{'badge_rhht': 'https://salt.tikicdn.com/ts/upload/51/ac/cc/528e80fe3f464f910174e2fdf8887b6f.png',
  'discount': '-7%',
  'freegift': False,
  'freeship': 'https://salt.tikicdn.com/ts/upload/dc/0d/49/3251737db2de83b74eba8a9ad6d03338.png',
  'image': 'https://salt.tikicdn.com/cache/200x200/ts/product/8b/7b/4d/b552547d3f21139e0bf413c60d7d7105.jpg',
  'installment': '',
  'name': 'Tai nghe chụp tai không dây Remax RB-660HB - Hàng chính hãng',
  'price': '649.000 ₫',
  'product_url': 'https://tka.tiki.vn/pixel/pixel?data=djAwMXkdlD29wCC4jXYM6taaTkNBTKLg6ZvaJ7-C9EmGEE8Pm6xoLgfsgbrPbA2NbIhv0hvFoCF848ERwuvVts82a0-DqU2eQuPTvVTROSLXLVY9yqFQ2UcAWpdNbVYrBEXQ1pvXEaxEwRQv6t8vDxw4JAZWh4b9ZanJ_ZCJvx22A9VpASMQxpqWpsYSVpJbrBbTsTftARmD_5LbxxutUpgDF5w1H3u3QrPA7Y7iUvXQKmfWd2T2qsklNCjUO7lXUJv66MTRuBZjrxVVNabETrVCeDwLzSfhPQH8DwIPBnsLf8RrekF2zsuIWBUYD5YePtLbtBWr2hMoAPAomAYokCP_oUhncTtVqyaZa9emfFAMyO3XuZtPFeJE72e0VUpxQF6DEaZJLGyVDqwAsHngHtuEJk5HudQfC3_sRRDTSw-lgbtae__y-a13y_1gUl7Ho7ReImmRsZ_T1sKu7bBEDRIsDz3x

# 3. Start scraping

Try to scrape at least 2 pages of your chosen main category.

In [ ]:
CHOOSEN = MAIN_CATEGORIES[2]
url = CHOOSEN['URL']
url2= 'https://tiki.vn/thiet-bi-kts-phu-kien-so/c1815?page=2&src=c.1815.hamburger_menu_fly_out_banner'
url3= 'https://tiki.vn/thiet-bi-kts-phu-kien-so/c1815?page=3&src=c.1815.hamburger_menu_fly_out_banner'


b=url2.index('&')
url2=url[:47]+"page=2"+url[52:]
url3=url[:47]+"page=3"+url[52:]
print(url)
print(url3)

https://tiki.vn/thiet-bi-kts-phu-kien-so/c1815?src=c.1815.hamburger_menu_fly_out_banner
https://tiki.vn/thiet-bi-kts-phu-kien-so/c1815?page=3.1815.hamburger_menu_fly_out_banner


In [ ]:
# Function to scrape all products from a page
def get_product_info_from_page(url3):
   
    global DRIVER

    data3 = []            # Store the info dictionary of each product in this list
    DRIVER.get(url3) # Use the driver to get info from the product page
    time.sleep(2)        # MUST have the sleep function

    all_products3 = DRIVER.find_elements(By.CLASS_NAME, 'product-item')
    print(type(len(all_products3)))
    print(f'Found {len(all_products3)} products')

    for product in all_products3:
      result = get_product_info_single(product) 
      data3.append(result)
    return data3

In [ ]:
url3=url[:47]+"page=3"+url[52:]
close_driver()
start_driver(force_restart=True)

DRIVER.get(url3)
print(DRIVER.current_url)
all_products3 = DRIVER.find_elements(By.CLASS_NAME,'product-item')
print(all_products3)
get_product_info_from_page('https://tiki.vn/thiet-bi-kts-phu-kien-so/c1815?page=3&src=c.1815.hamburger_menu_fly_out_banner')


https://tiki.vn/thiet-bi-kts-phu-kien-so/c1815?page=3.1815.hamburger_menu_fly_out_banner
[<selenium.webdriver.remote.webelement.WebElement (session="c77e0a51c9720ce9f3ff8dcd7cdcf991", element="7a55d74f-2aa1-486d-945d-7e50cfbb8343")>, <selenium.webdriver.remote.webelement.WebElement (session="c77e0a51c9720ce9f3ff8dcd7cdcf991", element="c65d4cad-f1a4-4439-82f0-43231329ae41")>, <selenium.webdriver.remote.webelement.WebElement (session="c77e0a51c9720ce9f3ff8dcd7cdcf991", element="bb7ed888-db64-4eda-b3bf-3126a49e06fc")>, <selenium.webdriver.remote.webelement.WebElement (session="c77e0a51c9720ce9f3ff8dcd7cdcf991", element="aab9b9b7-134e-47c9-993e-113855fe0f5f")>, <selenium.webdriver.remote.webelement.WebElement (session="c77e0a51c9720ce9f3ff8dcd7cdcf991", element="154db698-9e74-4bab-be22-035ad0eb9e3c")>, <selenium.webdriver.remote.webelement.WebElement (session="c77e0a51c9720ce9f3ff8dcd7cdcf991", element="e64e27eb-ea5f-4592-8250-73f2d0c4e63f")>, <selenium.webdriver.remote.webelement.WebEleme

/usr/local/lib/python3.7/dist-packages/selenium/webdriver/remote/webelement.py:341: UserWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  warnings.warn("find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead")


[{'badge_rhht': 'https://salt.tikicdn.com/ts/upload/51/ac/cc/528e80fe3f464f910174e2fdf8887b6f.png',
  'discount': False,
  'freegift': False,
  'freeship': 'https://salt.tikicdn.com/ts/upload/dc/0d/49/3251737db2de83b74eba8a9ad6d03338.png',
  'image': 'https://salt.tikicdn.com/cache/200x200/ts/product/4c/47/6c/a74d181570100f03aa1779f869604bb3.png',
  'installment': '',
  'name': 'Miếng sạc Energizer MagPAD 15W kèm cáp TypeC WCP119 - Thiết kế từ tính, siêu mỏng, sạc nhanh - HÀNG CHÍNH HÃNG',
  'price': '500.000 ₫',
  'product_url': 'https://tka.tiki.vn/pixel/pixel?data=djAwMVYGykwCXACkIXEFVUzRRv9j2xTH8sqOZvjVnlK8Y6jVJNxEL88CsdsuFftWJAQ_oBNLe8d1IJFANC8jIBd5c4h5B7MQyDqwBm5ElJIPMmBd4uxoQppNSwVqJg-eyetdAgZFlRn2EkPGb_2nlCg-VJFda-ZZQMveZC3QQ76Qzrom_wUFYYJ3I1jBUu2JccbHu1Kt8LcpHG-cG1mc565FPVpcQFKum3TQ6qpgYr_SoaHhjWxiMIO7nM4iX6RQ8OhFLHzWekHAXq0fX58lT5yKvvr3IDIiwS2zo1WP9PDz-zZnB1b-uUAhJK4McE08bV99j6Ed4DH8xwNTZyct_k9i-H5Rq1ylx4DucyFhElSVZpFg15iMWYq_E7kut-QyHtqswsEEEAWs5Q3ibgejhXF3-Kj3Xlz-JdMoI-IfFJ

In [ ]:
# def get_product_info_from_page(url):
#   DRIVER.get(url)
#   print(DRIVER.current_url)
#   products_all = DRIVER.find_elements(By.CLASS_NAME,'product-item')
#   print(products_all)
#   data1= get_product_info_single(product)
#   return data1

# def get_product_info_from_page(url3):
#   start_driver(force_restart=True)
#   DRIVER.get(url3)
#   print(DRIVER.current_url)
#   all_products3 = DRIVER.find_elements(By.CLASS_NAME,'product-item')
#   print(all_products3)
#   data3= get_product_info_single(product)
#   return data3

In [ ]:
data3= get_product_info_from_page('https://tiki.vn/thiet-bi-kts-phu-kien-so/c1815?page=3&src=c.1815.hamburger_menu_fly_out_banner')


<class 'int'>
Found 48 products


/usr/local/lib/python3.7/dist-packages/selenium/webdriver/remote/webelement.py:341: UserWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  warnings.warn("find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead")


In [ ]:

main_cat  = MAIN_CATEGORIES[2] # Pick any category you like by changing the index

start_driver(force_restart=True)
print('Scraping', main_cat['Name'])
print('Link:', main_cat['URL'])

# prod_data = [] # STORE YOUR PRODUCT INFO DICTIONARIES IN HERE
url= main_cat['URL']
url3=url[:47]+"page=3"+url[52:]
print('Link:', url3)

prod_data={}
prod_data=data3.copy()

print(prod_data)

close_driver() # Close driver when we're done

Scraping Phụ Kiện - Thiết Bị Số
Link: https://tiki.vn/thiet-bi-kts-phu-kien-so/c1815?src=c.1815.hamburger_menu_fly_out_banner
Link: https://tiki.vn/thiet-bi-kts-phu-kien-so/c1815?page=3.1815.hamburger_menu_fly_out_banner
[{'name': 'Chuột có dây Prolink PMC2002 - Thiết kế Ambidextrous (đối xứng) phù hợp với người thuận tay trái lẫn tay phải, dùng cho Laptop, Notebook, PC và Mac - Hàng chính hãng', 'price': '83.000 ₫', 'discount': '-31%', 'product_url': 'https://tka.tiki.vn/pixel/pixel?data=djAwMfpfHSSMiIXEBAOltFwr29kV1vV6nxntUj-uZZImkDCoOVfS82FQpklQARnrwy-T0aHKSnc4kh3FlPy_uYdkq_9Ai0ll2hh_LQq1kTfSzEgPHR_XgwwwObc5CIwM2hp-2nfGPoLUm8YTIk5Y7C_LuyW45SXqrKOupLuD_IVQqr5wjp0zmYEPscobNzMD9AsiAjziS3mzyX8SMSGIxuaAqizzeVJWPrFTn00Ll-aqiEIEKIXd6C1ZFl9Yp2CtpaYBvfJw_cnRKYS2Tj0gpBXhsWTpxMy0x7dkyQtVTrQ0aEvaHkmZGH5oG8kiHeWFzVuuNFAjWRRA40rm07SpXuqReUlm86P4HQHyEpgjT5P_UpItN_OnrN9OYD7C_s8kxqB5rTT_uH2XUoq3-HQa-fhRxlD9p4hmZqKfXS5Hh4s-jJe24GBBH_aEalDTDtUKZwBZAqVrkhv93zNft-3aPnEI9-rP6EPp4dpYkOCya_VC2iDVbQ2ljlvSTn

# 4. Run cell below to save your scraped data into a .csv file
If you've scraped correctly, then the cell should run without error and the information in the table should look reasonable.

In [ ]:
# SAVE DATA TO CSV FILE
df = pd.DataFrame(data=prod_data, columns=prod_data[0].keys())
df.to_csv('tiki_products.csv')

n_products_to_view = 50 # Change this as you like to check more products
df.head(n_products_to_view)

,name,price,discount,product_url,image,freeship,rating,badge_rhht,freegift,installment
0,Chuột có dây Prolink PMC2002 - Thiết kế Ambide...,83.000 ₫,-31%,https://tka.tiki.vn/pixel/pixel?data=djAwMfpfH...,https://salt.tikicdn.com/cache/200x200/ts/prod...,https://salt.tikicdn.com/ts/upload/dc/0d/49/32...,4.7,False,False,
1,HÚT BỤI BÀN PHÍM LAPTOP MINI,52.500 ₫,-25%,https://tiki.vn/hut-bui-ban-phim-laptop-mini-p...,https://salt.tikicdn.com/cache/200x200/ts/prod...,https://salt.tikicdn.com/cache/200x200/ts/prod...,0.0,False,False,
2,"Đế tản nhiệt laptop, máy tính xách tay H9 có đ...",263.000 ₫,-2%,https://tiki.vn/de-tan-nhiet-laptop-may-tinh-x...,https://salt.tikicdn.com/cache/200x200/ts/prod...,https://salt.tikicdn.com/ts/upload/dc/0d/49/32...,4.6,False,False,
3,[ Xuất Khẩu Mỹ ] - Webcam Full HD 1K/2K/4K Cam...,4.900.000 ₫,-2%,https://tiki.vn/xuat-khau-my-webcam-full-hd-1k...,https://salt.tikicdn.com/cache/200x200/ts/prod...,https://salt.tikicdn.com/cache/200x200/ts/prod...,5.0,False,False,
4,Thẻ nhớ MicroSD Kioxia 32GB Class 10 - Hàng Nh...,90.000 ₫,-3%,https://tiki.vn/the-nho-microsd-kioxia-32gb-cl...,https://salt.tikicdn.com/cache/200x200/ts/prod...,https://salt.tikicdn.com/ts/upload/e0/41/da/bb...,4.5,https://salt.tikicdn.com/ts/upload/9f/32/dd/8a...,False,
5,Ốp UAG Metropolis SE cho Microsoft Surface Pro...,1.700.000 ₫,False,https://tiki.vn/op-uag-metropolis-se-cho-micro...,https://salt.tikicdn.com/cache/200x200/ts/prod...,https://salt.tikicdn.com/ts/upload/b9/1f/4b/55...,0.0,False,False,
6,"Sim 12FD50, 4GB/ngày sử dụng 12 tháng của Mobi...",420.000 ₫,False,https://tiki.vn/sim-12fd50-4gb-ngay-su-dung-12...,https://salt.tikicdn.com/cache/200x200/ts/prod...,https://salt.tikicdn.com/ts/upload/dc/0d/49/32...,5.0,https://salt.tikicdn.com/ts/upload/51/ac/cc/52...,False,
7,Củ Sạc Nhanh PD 20W Remax cổng Type C - Dành C...,368.900 ₫,-9%,https://tiki.vn/cu-sac-nhanh-pd-20w-remax-cong...,https://salt.tikicdn.com/cache/200x200/ts/prod...,https://salt.tikicdn.com/ts/upload/dc/0d/49/32...,0.0,False,False,
8,Bộ chia USB 1 ra 4 cổng 3.0 - Hub USB 4 Port 3.0,53.990 ₫,False,https://tiki.vn/bo-chia-usb-1-ra-4-cong-3-0-hu...,https://salt.tikicdn.com/cache/200x200/ts/prod...,https://salt.tikicdn.com/ts/upload/dc/0d/49/32...,4.7,https://salt.tikicdn.com/ts/upload/51/ac/cc/52...,False,
9,Bộ Núm Silicone Thay Thế Dành Cho Tai Nghe Air...,79.000 ₫,False,https://tiki.vn/bo-num-silicone-thay-the-cho-t...,https://salt.tikicdn.com/cache/200x200/ts/prod...,https://salt.tikicdn.com/ts/upload/dc/0d/49/32...,0.0,False,False,


# OPTIONAL: Extra information


If you've managed to successfully completed all of the above, you can look to get extra information for each product.

* Does it has FreeShip? <img src="https://salt.tikicdn.com/ts/upload/dc/0d/49/3251737db2de83b74eba8a9ad6d03338.png">
* Number of reviews?
* Number of sale?
* How many stars or percentage of stars?
* Does it got "badge under price" (Rẻ hơn hoàn tiền)? <img src="https://salt.tikicdn.com/ts/upload/51/ac/cc/528e80fe3f464f910174e2fdf8887b6f.png">
* Discount percentage?
* Does it got "shocking price" badge? <img src="https://salt.tikicdn.com/ts/upload/75/34/d2/4a9a0958a782da8930cdad8f08afff37.png">
* Does it allowed to be paid by installments? <img src="https://salt.tikicdn.com/ts/upload/ba/4e/6e/26e9f2487e9f49b7dcf4043960e687dd.png">
* Does it comes with free gifts? <img src="https://salt.tikicdn.com/ts/upload/47/35/8c/446f61d046eba9a305d3f39dc0834c4a.png">
    
